# 함수 호출

# 개요
OpenAI의 함수 호출 기능은 AI가 생성한 텍스트를 외부 함수와 통합할 수 있도록 합니다. \
이를 통해 모델의 출력에 따라 코드를 실행하거나 계산을 수행하거나 데이터를 가져올 수 있습니다. \
함수를 정의하고 모델이 이를 호출하도록 허용함으로써 더 상호작용적이고 동적인 애플리케이션을 만들 수 있습니다.

더 자세한 예제는 공식 문서를 참조하세요: [Azure OpenAI 서비스의 함수 호출](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling?tabs=non-streaming%2Cpython)



In [1]:
import os
from openai import AzureOpenAI
import json
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)
CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

## 인터넷에서 정보를 검색하는 등의 작업을 수행할 함수를 정의합니다.

In [ ]:
import requests

# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="celsius"):
    """
    Fetch current weather using the wttr.in public API (no API key required).
    """
    # wttr.in supports city names directly in the URL
    # unit: 'm' for metric (Celsius), 'u' for US (Fahrenheit)
    unit_flag = 'u' if unit == "fahrenheit" else 'm'
    url = f"https://wttr.in/{location}?format=j1&{unit_flag}"
    try:
        resp = requests.get(url, timeout=5)
        resp.raise_for_status()
        data = resp.json()
        # wttr.in returns temperature in Celsius by default
        temp_c = data["current_condition"][0]["temp_C"]
        temp_f = data["current_condition"][0]["temp_F"]
        city = location
        temperature = temp_f if unit == "fahrenheit" else temp_c
        return json.dumps({"location": city, "temperature": temperature, "unit": unit})
    except Exception as e:
        return json.dumps({"location": location, "temperature": "unknown", "error": str(e)})



## 질문을 해봅시다

In [6]:
def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role":"system", "content":"""귀하는 사용자가 날씨에 대한 정보를 얻을 수 있도록 설계된 AI 어시스턴트입니다. 
                 사용자가 날씨에 대한 정보를 요청하면 'get_current_weather'라는 도구 함수를 사용해야 합니다. 
                 제공된 함수만 사용하세요.
                 함수에 어떤 값을 입력할지 함부로 추측하지 마세요. 
                 사용자 요청이 모호한 경우 설명을 요청하세요.
                 인수의 형식은 '/*'를 포함해서는 안 되며, 속성 이름이 큰따옴표로 묶인 유효한 JSON 문자열이어야 합니다."""},
                 {"role": "user", "content": "서울 마포구의 날씨는 어떤가요?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=messages,
        tools=tools,
        tool_choice = {"type": "function", "function": {"name": "get_current_weather"}}
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            # Sometimes the name is classified mistakenly as python
            if function_name == "python":
                function_name = "get_current_weather"
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
    


print(run_conversation().choices[0].message.content)

서울 마포구의 현재 온도는 15도입니다.


#### 탐색할 사용 사례
1. **동적 계산**  \
사용자 입력에 따라 실시간으로 계산을 수행합니다.

2. **데이터 가져오기** \
모델의 출력에 따라 외부 API에서 데이터를 가져옵니다.

3. **상호작용 애플리케이션** \
AI와 백엔드 함수를 통합하여 더 상호작용적이고 동적인 애플리케이션을 만듭니다.

#### 모범 사례
1. **명확한 프롬프트** \
모델로부터 정확한 응답을 얻기 위해 프롬프트를 명확하고 구체적으로 작성하세요.

2. **견고한 파싱** \
모델 출력의 다양한 형식을 처리할 수 있도록 견고한 파싱 로직을 구현하세요.

3. **오류 처리** \
예상치 못한 출력이나 함수 호출 실패를 관리하기 위해 오류 처리를 포함하세요.